In [1]:
from imblearn.metrics import geometric_mean_score
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn import metrics
np.random.seed(13)
import time
from datetime import datetime, timedelta
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn import preprocessing


import ast


In [2]:
def smote(x,y):
    X_resampled, y_resampled = SMOTE().fit_sample(x, y)
    #print('check',sum(y_resampled)/len(y_resampled))
    return X_resampled,y_resampled
def balance(x,y):
    posindex=np.where( y == 1 )
    negindex=np.where( y == 0 )
    xt=[]
    yt=[]
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    yt=np.concatenate((y[posindex[0][0:nindex]],y[negindex[0][0:nindex]]))
    xt=np.concatenate((x[posindex[0][0:nindex]],x[negindex[0][0:nindex]]))
    
    return xt,yt

def balanceup(x,y):
    posindex=np.where( y == 1 )
    negindex=np.where( y == 0 )
    xt=[]
    yt=[]
    yindex=[]
    
    if(len(posindex[0])!=0 and len(negindex[0])!=0):
       
        nindex=max(len(posindex[0]),len(negindex[0]))
        mini=min(len(posindex[0]),len(negindex[0]))
        diff=nindex-mini
        u=0
        for i in range(0,mini):
            yt.append(y[posindex[0][i]])
            yt.append(y[negindex[0][i]])
            xt.append(x[posindex[0][i]])
            xt.append(x[negindex[0][i]])
        #print('first',sum(yt)/len(yt)) 
        if(len(posindex[0])>len(negindex[0])):
            toextract=negindex
            enter=posindex
        else:
            toextract=posindex
            enter=negindex
        if(diff!=0 and len(toextract[0])!=0):
            for i in range(0,diff):
                r=np.random.randint(0,len(toextract))
                yt.append(y[toextract[0][r]])
                xt.append(x[toextract[0][r]])
                yt.append(y[enter[0][mini+i]])
                xt.append(x[enter[0][mini+i]])
    else:
        #print('Unbalance')
        u=1
        xt=x
        yt=y
    #print(sum(yt)/len(yt))              
    return xt,yt,u
def expercentage(y,prediction,percentage,t):
    tot=0
    falsetotpos=0
    falsetotneg=0
    for i in range(0,len(percentage)-1):
        if(prediction[i]==1):
            tot=tot+percentage[i]
        if(prediction[i]==0):
            tot=tot-percentage[i]
            
        falsetotpos=falsetotpos+percentage[i]
        falsetotneg=falsetotneg-percentage[i]
        i=i+t
    return tot,falsetotneg, falsetotpos

In [ ]:
# ==========    SVM + GOOGLE SENTIMENT   =================
#
# EXPERIMENT WITH TIMESTEP 1H
# EXPERIMENT WITH TIMESTEP 15MIN googleSentPerTimestep15min.csv
#

news =  pd.read_csv('/home/simone/Desktop/NLFF/GoogleNL/googleSentPerTimestep.csv')
price = pd.read_csv('/home/simone/Desktop/NLFF/indexes/indexesAAPL.csv')
#price = pd.read_csv('/home/simone/Desktop/NLFF/TechnicalDatasetUpdated/AAPL.csv')

price = price.rename(index=str, columns={"date": "DATE"})
news = news.rename(index=str, columns={"initTime": "DATE"})

news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in news['DATE']]
# This datased is already GMT+0
price['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in price['DATE']]
news['entSentVec'] = [ast.literal_eval(row) for row in news['entSentVec']]

#ALLIGNMENT
initDate = news['DATE'][0]
finalDate = news['DATE'][len(news)-1]

price.drop(price[price.DATE > finalDate].index, inplace=True)
price.drop(price[price.DATE < initDate].index, inplace=True)
price = price.reset_index(drop=True)

for data in news['DATE']:
    if data not in price['DATE'].tolist():
        print(data)

assert len(price) == len(news)



sentiment = np.asarray(news['entSentVec'].tolist())
sentiment = pd.DataFrame(data=sentiment)
meanvector = pd.concat([sentiment, sentiment.rolling(5).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(10).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(15).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(20).mean()],axis=1)


print(meanvector.shape)

# prices=list()
# sentiment=list()
# for file in files:
#     sentimentVector =  pd.read_csv('/home/andrea/Desktop/NLFF/intrinioDatasetUpdated/Sentiment/'+file+'.csv')
#     price= pd.read_csv('/home/andrea/Desktop/NLFF/TechnicalDatasetUpdated/'+file+'.csv')
#     #alignment of data
#     maxdata=max(sentimentVector['initTime'])
#     mindata=min(sentimentVector['initTime'])
#     price['Unnamed: 0'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in price['Unnamed: 0']]
#     price['Unnamed: 0'] = [row-timedelta(hours=8) for row in price['Unnamed: 0']]
#     price=price[price['Unnamed: 0']>=mindata]
#     price=price[price['Unnamed: 0']<=maxdata]
#     print(sentimentVector[0:10])
#     print(price[0:10])
#     sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)
#     meanvector=np.concatenate((sentimentVector,pd.rolling_mean(sentimentVector,2)),axis=1)
#     meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,4)),axis=1)
#     meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,6)),axis=1)
#     meanvector=np.concatenate((meanvector,pd.rolling_mean(sentimentVector,8)),axis=1)
#     price=price.drop(['Unnamed: 0'],axis=1)
#     #['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
#     #df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
#     #df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
#     #
#     sentiment.append(meanvector)
#     prices.append(price)

In [3]:
# ==========  SVM + LOUGRAM DICTIONARY /!\ ALTERNATIVO ALLA SEZIONE PRECEDENTE  =================
#
# EXPERIMENT WITH TIMESTEP 1H
# EXPERIMENT WITH TIMESTEP 15MIN googleSentPerTimestep15min.csv
#

news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentFullAggregatedHourly/AAPL.csv')
price = pd.read_csv('/home/simone/Desktop/NLFF/indexes/indexesAAPL.csv')
#price = pd.read_csv('/home/simone/Desktop/NLFF/TechnicalDatasetUpdated/AAPL.csv')

price = price.rename(index=str, columns={"date": "DATE"})
news = news.rename(index=str, columns={"initTime": "DATE"})

news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in news['DATE']]
# This datased is already GMT+0
price['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in price['DATE']]

#ALLIGNMENT
initDate = datetime(2018, 1, 2, 0, 0, 0)
finalDate = datetime(2018, 3, 31, 0, 0, 0)

news.drop(news[news.DATE > finalDate].index, inplace=True)
news.drop(news[news.DATE < initDate].index, inplace=True)
news = news.reset_index(drop=True)
price.drop(price[price.DATE > finalDate].index, inplace=True)
price.drop(price[price.DATE < initDate].index, inplace=True)
price = price.reset_index(drop=True)

for data in news['DATE']:
    if data not in price['DATE'].tolist():
        print(data)

assert len(price) == len(news)


sentiment = news.drop(['Unnamed: 0', 'DATE'], axis=1)

meanvector = pd.concat([sentiment, sentiment.rolling(5).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(10).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(15).mean()],axis=1)
meanvector = pd.concat([meanvector, sentiment.rolling(20).mean()],axis=1)


print(meanvector.shape)


KeyError: 'DATE'

In [ ]:
import warnings
warnings.filterwarnings('ignore')
chartlist=[]
gainlist=[]

   

senttemp=np.nan_to_num(np.asarray(meanvector, dtype=float))
senttemp=normalize(senttemp,axis=0,norm='max')

percpostotal=[]
percnegtotal=[]
TREND_WINDOW = 4


x=[]
y=[]
percentage=[]

yvolatility=[]
#print('============================================================')
#print('Working on window:',t)
#print(len(xtemp))
##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
up = 0
down = 0
for i in range(0,len(price)-TREND_WINDOW-1):
    s=np.sign(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open'])
    percentage.append((100*(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open']))/price.iloc[i+1]['open']) 
    if(s==-1):
        y.append(0)
        down +=1
    else:
        y.append(1)
        up +=1
    yvolatility.append((100*abs(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open']))/price.iloc[i+1]['open'])    
    x.append(senttemp[i])
    
print('UP: '+str(up)+'\t DOWN: '+str(down))

y=np.array(y)
x=np.array(x)
percentage=np.array(percentage)
permindex=range(0,len(x))
#permindex=np.random.permutation(permindex)
train=0.8
nt=math.ceil(len(x)*train)
trainvalindex=permindex[0:nt]
testindex=permindex[nt:]

yvolatility=np.array(yvolatility)
x_tv=[]
y_tv=[]
x_test=[]
y_test=[]
x_tv=x[trainvalindex]
y_tv=y[trainvalindex]
x_test=x[testindex]
y_test=y[testindex]
print('UP TRAIN-VAL: '+str(sum(y_tv)/len(y_tv)*100)+'%')
print('UP TEST: '+str(sum(y_test)/len(y_test)*100)+'%')

yvolatilitytest=yvolatility[testindex]
#create structure for percentile valuation
distribution=[]

step=(max(yvolatilitytest)-min(yvolatilitytest))/5
for v in range(0,5):
    pindexes=[]
    for r in range(0,len(y_test)):
        if(yvolatilitytest[r]<min(yvolatilitytest)+step+step*v and yvolatilitytest[r]>min(yvolatilitytest)+step*v):
            pindexes.append(r)
    distribution.append(pindexes)


cspace=np.logspace(-4,4,10)
bestsvm=None
maxacc=0
cvacc=0
maxc=0

totu=0
for c in cspace:
#print()

    cvacclist=[]
    #faccio cross validation
    #start with 40% as train and 10% for validation and then i move in percentege
    # 0-40 40-50
    # 0-50 50-60
    # 0-60 60-70 
    #etc test set is completely external i do in some way error extimation changin the ticker
    trainpoint=math.floor(len(x_tv)*0.40)
    dimval=math.floor(trainpoint*0.25)
    endval=trainpoint+dimval

    for i in range(0,6):
        x_train=x_tv[0:trainpoint]
        y_train=y_tv[0:trainpoint]
        x_val=x_tv[trainpoint:endval]
        y_val=y_tv[trainpoint:endval]

        trainpoint=trainpoint+dimval
        endval=endval+dimval
        p=sum(y_train)/(len(y_train)-sum(y_train))
        rbf_svm=svm.SVC(kernel='linear',C=c)
        
        x_train,y_train=smote(x_train,y_train)
        rbf_svm.fit(x_train,y_train)


        if(sum(y_val)+6<len(x_val) and sum(y_val)>6):
            x_val,y_val=smote(x_val,y_val)
        else:
            totu=totu+1
        prediction=rbf_svm.predict(x_val)

        cvacclist.append(rbf_svm.score(x_val,y_val))

    cvacc=sum(cvacclist)/len(cvacclist)
    if(cvacc>maxacc):
        maxacc=cvacc
        maxc=c



p=sum(y_tv)/(len(y_tv)-sum(y_tv))
rbf_svm=svm.SVC(kernel='linear',C=maxc)
x_tv,y_tv=smote(x_tv,y_tv)
rbf_svm.fit(x_tv,y_tv)
prediction=rbf_svm.predict(x_test)
volr=[]
voln=[]
for j in range(0,len(x_test)):
    if(y_test[j]==prediction[j]):
        volr.append(yvolatilitytest[j])
    else:
        voln.append(yvolatilitytest[j])
confmatrix=confusion_matrix(y_test, prediction)
CM = f1_score(y_test, prediction)
fpr, tpr, thresholds = metrics.roc_curve(y_test, prediction)
auc=metrics.auc(fpr, tpr)
valueacc=rbf_svm.score(x_test,y_test)

print('F1,score',CM)
print('Acc-score:',valueacc)
print('Window:',TREND_WINDOW)
print(confmatrix)
print('Vol pos',sum(volr)/len(volr))
print('Vol neg',sum(voln)/len(voln))
percnegtotal.append(sum(volr)/len(volr))
percpostotal.append(sum(voln)/len(voln))
#build structure to plot distribution onver percentile
chart=np.zeros(5)
for r in range(0,len(y_test)):
    for v in range(0,5):
        if(yvolatilitytest[r]<min(yvolatility)+step+step*v and yvolatilitytest[r]>min(yvolatility)+step*v):
            chart[v]+=1
plt.plot(chart)
plt.title('How many samples in each bin of movements')

plt.show()

print('Percentage of gain:',(sum(volr)-sum(voln))/(len(volr)+len(voln)))
print('Total gain:',len(y_test)*(sum(volr)-sum(voln))/(len(volr)+len(voln)))
gainlist.append(len(y_test)*(sum(volr)-sum(voln))/(len(volr)+len(voln)))
print('===============================================')
#performances for each percentile
chartscore=[]
for g in range(0,len(distribution)):
    print('Percentile: ',g)
    tempscore=0
    if(len(distribution[g])>0):
        predictionp=rbf_svm.predict(x_test[distribution[g]])
        print(confusion_matrix(y_test[distribution[g]], predictionp))
        print(rbf_svm.score(x_test[distribution[g]],y_test[distribution[g]]))
        tempscore=rbf_svm.score(x_test[distribution[g]],y_test[distribution[g]])
    chartscore.append(tempscore)    
chartlist.append(chartscore)
print('===============================================')
print('Mean gain over portfolio : ', sum(gainlist)/len(gainlist))
#print all the accuracy varing the percentile
for st in range(0,len(chartlist)):
    plt.plot(chartlist[st])
    plt.title("Acciuracies in various bins of movements ")
    
plt.show()
        
        
        

In [ ]:
# GOOGLE, 15 min, trend = 4

# UP: 845	 DOWN: 797
# UP TRAIN-VAL: 53.50076103500761%
# UP TEST: 43.292682926829265%
# F1,score 0.41877256317689526
# Acc-score: 0.5091463414634146
# Window: 4
# [[109  77]
#  [ 84  58]]
# ====
# Percentile:  0
# [[49 37]
#  [43 30]]
# 0.4968553459119497
# Percentile:  1
# [[45 16]
#  [31  7]]
# 0.5252525252525253
# Percentile:  2
# [[ 9 11]
#  [ 7 11]]
# 0.5263157894736842
# Percentile:  3
# [[4 8]
#  [3 8]]
# 0.5217391304347826
# Percentile:  4
# [[1 5]
#  [0 0]]
# 0.16666666666666666

# =======
# Mean gain over portfolio :  -1.6690018545109666


# GOOGLE, 1h, trend = 4

# UP: 219 DOWN: 203
# UP TRAIN-VAL: 56.50887573964497%
# UP TEST: 33.33333333333333%
# F1,score 0.5148514851485149
# Acc-score: 0.4166666666666667
# Window: 4
# [[ 9 47]
#  [ 2 26]]
# ===============================================
# Percentile:  0
# [[ 3 22]
#  [ 1 10]]
# 0.3611111111111111
# Percentile:  1
# [[ 1 16]
#  [ 0  9]]
# 0.38461538461538464
# Percentile:  2
# [[1 4]
#  [1 3]]
# 0.4444444444444444
# Percentile:  3
# [[2 4]
#  [0 3]]
# 0.5555555555555556
# Percentile:  4
# [[1 0]
#  [0 1]]
# 1.0
# ===============================================
# Mean gain over portfolio :  -1.0599238818218666



# DICTIONARY, 15min, trend = 4

# UP: 845	 DOWN: 797
# UP TRAIN-VAL: 53.50076103500761%
# UP TEST: 43.292682926829265%
# F1,score 0.5626740947075208
# Acc-score: 0.5213414634146342
# Window: 4
# [[ 70 116]
#  [ 41 101]]
# ===============================================
# Percentile:  0
# [[33 53]
#  [23 50]]
# 0.5220125786163522
# Percentile:  1
# [[24 37]
#  [11 27]]
# 0.5151515151515151
# Percentile:  2
# [[ 3 17]
#  [ 5 13]]
# 0.42105263157894735
# Percentile:  3
# [[ 7  5]
#  [ 1 10]]
# 0.7391304347826086
# Percentile:  4
# [[3 3]
#  [0 0]]
# 0.5
# ===============================================
# Mean gain over portfolio :  10.505373716586945



# DICTIONARY, 1H, trend = 4

# UP: 219	 DOWN: 203
# UP TRAIN-VAL: 56.50887573964497%
# UP TEST: 33.33333333333333%
# F1,score 0.32558139534883723
# Acc-score: 0.6547619047619048
# Window: 4
# [[48  8]
#  [21  7]]
# ===============================================
# Percentile:  0
# [[20  5]
#  [ 8  3]]
# 0.6388888888888888
# Percentile:  1
# [[15  2]
#  [ 5  4]]
# 0.7307692307692307
# Percentile:  2
# [[4 1]
#  [4 0]]
# 0.4444444444444444
# Percentile:  3
# [[6 0]
#  [3 0]]
# 0.6666666666666666
# Percentile:  4
# [[1 0]
#  [1 0]]
# 0.5
# ===============================================
# Mean gain over portfolio :  31.90151202294833





In [ ]:
# ============== SVR (Regression) ========================

import warnings
warnings.filterwarnings('ignore')
chartlist=[]
gainlist=[]


senttemp=np.nan_to_num(np.asarray(meanvector, dtype=float))
senttemp=normalize(senttemp,axis=0,norm='max')

percpostotal=[]
percnegtotal=[]
TREND_WINDOW = 4


x=[]
y=[]
percentage=[]

yvolatility=[]
#print('============================================================')
#print('Working on window:',t)
#print(len(xtemp))
##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
for i in range(0,len(price)-TREND_WINDOW-1):
    y.append(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open'])
    percentage.append((100*(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open']))/price.iloc[i+1]['open']) 
    yvolatility.append((100*abs(price.iloc[i+TREND_WINDOW+1]['close']-price.iloc[i+1]['open']))/price.iloc[i+1]['open'])    
    x.append(senttemp[i])
    
y=np.array(y)
x=np.array(x)
percentage=np.array(percentage)
permindex=range(0,len(x))
#permindex=np.random.permutation(permindex)
train=0.8
nt=math.ceil(len(x)*train)
trainvalindex=permindex[0:nt]
testindex=permindex[nt:]

yvolatility=np.array(yvolatility)
x_tv=[]
y_tv=[]
x_test=[]
y_test=[]
x_tv=x[trainvalindex]
y_tv=y[trainvalindex]
x_test=x[testindex]
y_test=y[testindex]

yvolatilitytest=yvolatility[testindex]
#create structure for percentile valuation
distribution=[]

step=(max(yvolatilitytest)-min(yvolatilitytest))/5
for v in range(0,5):
    pindexes=[]
    for r in range(0,len(y_test)):
        if(yvolatilitytest[r]<min(yvolatilitytest)+step+step*v and yvolatilitytest[r]>min(yvolatilitytest)+step*v):
            pindexes.append(r)
    distribution.append(pindexes)


cspace=np.logspace(-4,4,10)
bestsvm=None
loss= float("inf")
cvacc=0
best_c=0

totu=0
for c in cspace:
#print()

    cvacclist=[]
    #faccio cross validation
    #start with 40% as train and 10% for validation and then i move in percentege
    # 0-40 40-50
    # 0-50 50-60
    # 0-60 60-70 
    #etc test set is completely external i do in some way error extimation changin the ticker
    trainpoint=math.floor(len(x_tv)*0.40)
    dimval=math.floor(trainpoint*0.25)
    endval=trainpoint+dimval

    for i in range(0,6):
        x_train=x_tv[0:trainpoint]
        y_train=y_tv[0:trainpoint]
        x_val=x_tv[trainpoint:endval]
        y_val=y_tv[trainpoint:endval]

        trainpoint=trainpoint+dimval
        endval=endval+dimval
        p=sum(y_train)/(len(y_train)-sum(y_train))
        rbf_svm=svm.SVR(kernel='linear',C=c)
        rbf_svm.fit(x_train,y_train)

        prediction=rbf_svm.predict(x_val)

        cvacclist.append(rbf_svm.score(x_val,y_val))

    cvloss=sum(cvacclist)/len(cvacclist)
    if(cvloss<loss):
        loss=cvloss
        best_c=c



p=sum(y_tv)/(len(y_tv)-sum(y_tv))
rbf_svm=svm.SVR(kernel='linear',C=best_c)
rbf_svm.fit(x_tv,y_tv)
prediction=rbf_svm.predict(x_test)
volr=[]
voln=[]
for j in range(0,len(x_test)):
    if(y_test[j]==prediction[j]):
        volr.append(yvolatilitytest[j])
    else:
        voln.append(yvolatilitytest[j])
confmatrix=confusion_matrix(np.sign(y_test), np.sign(prediction))
CM = f1_score(np.sign(y_test), np.sign(prediction))
fpr, tpr, thresholds = metrics.roc_curve(np.sign(y_test), np.sign(prediction))
auc=metrics.auc(fpr, tpr)
y_pred=rbf_svm.predict(x_test)

print('F1-score',CM)
print('MSE-score:',((y_test - y_pred) ** 2).sum() )
print('Window:',TREND_WINDOW)
print(confmatrix)
print('Acc:',((confmatrix[0][0]+confmatrix[1][1])/(np.sum(np.asarray(confmatrix)))))

In [ ]:
plt.plot(y_test)
plt.plot(y_pred, color='r')